# neural network practice

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## load and prepare the data
make variables all the same weight

In [ ]:
data_path = 'Bike-Sharing-Dataset/hour.csv'

rides = pd.read_csv(data_path)

In [ ]:
rides.head()

## the data

number of bike riders over the first 10 days

In [ ]:
rides[:24*10].plot(x='dteday', y='cnt')

### dummy variables
binary dummy variables

In [ ]:
dummy_fields = ['season', 'weathersit', 'mnth', 'hr', 'weekday']
for each in dummy_fields:
    dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
    rides = pd.concat([rides, dummies], axis=1)

fields_to_drop = ['instant', 'dteday', 'season', 'weathersit', 
                  'weekday', 'atemp', 'mnth', 'workingday', 'hr']
data = rides.drop(fields_to_drop, axis=1)
data.head()

### scaling target variables
zero mean; standard deviation of 1

In [ ]:
quant_features = ['casual', 'registered', 'cnt', 'temp', 'hum', 'windspeed']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std

### splitting the data into training, testing, and validation sets
last approximately 21 days = test set 

In [ ]:
# save data for the last 21 days 
test_data = data[-21*24:]

# removes the test data from the data set 
data = data[:-21*24]

# separate the data into features and targets
target_fields = ['cnt', 'casual', 'registered']
features, targets = data.drop(target_fields, axis=1), data[target_fields]
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]

In [ ]:
# last 60 days = validation set
train_features, train_targets = features[:-60*24], targets[:-60*24]
val_features, val_targets = features[-60*24:], targets[-60*24:]

## building the network

Each layer of our network calculates the outputs for each neuron and the outputs from one layer become inputs to the neurons on the next layer. = forward propagation

We use the weights to propagate signals forward from the input to the output layers in a neural network and to also propagate error backwards from the output back into the network to update the weights = backpropagation

In [ ]:
# TODO: replace these values with calculations
#set nodes
self.input_nodes = input_nodes
self.hidden_nodes = hidden_nodes
self.output_nodes = output_nodes

# Initialize weights
self.weights_input_to_hidden = np.random.normal(0.0, self.input_nodes**-0.5,(self.input_nodes, self.hidden_nodes))
self.weights_hidden_to_output = np.random.normal(0.0, self.hidden_nodes**-0.5,(self.hidden_nodes, self.output_nodes))
self.lr = learning_rate
        
self.activation_function = lambda x : 0  # Replace 0 with the sigmoid calculation.

# train
n_records = features.shape[0]
delta_weights_i_h = np.zeros(self.weights_input_to_hidden.shape)
delta_weights_h_o = np.zeros(self.weights_hidden_to_output.shape)
for X, y in zip(features, targets):
    final_outputs, hidden_outputs = self.forward_pass_train(X)  #forward pass function
    delta_weights_i_h, delta_weights_h_o = self.backpropagation(final_outputs, hidden_outputs, X, y, delta_weights_i_h, delta_weights_h_o)
    self.update_weights(delta_weights_i_h, delta_weights_h_o, n_records)

# forward pass
hidden_inputs = None # signals into hidden layer
hidden_outputs = None # signals from hidden layer

final_inputs = None # signals into final output layer
final_outputs = None # signals from final output layer

# back propagation

error = None # Output layer error is the difference between desired target and actual output.
        
hidden_error = None 
output_error_term = None       
hidden_error_term = None
        
delta_weights_i_h += None
delta_weights_h_o += None

# update weights

self.weights_hidden_to_output += None # update hidden-to-output weights with gradient descent step
self.weights_input_to_hidden += None # update input-to-hidden weights with gradient descent step

# forward pass

hidden_inputs = None # signals into hidden layer
hidden_outputs = None # signals from hidden layer
        
final_inputs = None # signals into final output layer
final_outputs = None # signals from final output layer 

In [ ]:
def MSE(y, Y):
    return np.mean((y-Y)**2)

In [ ]:
import sys

iterations = 100
learning_rate = 0.1
hidden_nodes = 2
output_nodes = 1

N_i = train_features.shape[1]
network = NeuralNetwork(N_i, hidden_nodes, output_nodes, learning_rate)

losses = {'train':[], 'validation':[]}
for ii in range(iterations):
    # Go through a random batch of 128 records from the training data set
    batch = np.random.choice(train_features.index, size=128)
    X, y = train_features.ix[batch].values, train_targets.ix[batch]['cnt']
                             
    network.train(X, y)
    
    # Printing out the training progress
    train_loss = MSE(network.run(train_features).T, train_targets['cnt'].values)
    val_loss = MSE(network.run(val_features).T, val_targets['cnt'].values)
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations)) \
                     + "% ... Training loss: " + str(train_loss)[:5] \
                     + " ... Validation loss: " + str(val_loss)[:5])
    sys.stdout.flush()
    
    losses['train'].append(train_loss)
    losses['validation'].append(val_loss)

In [ ]:
plt.plot(losses['train'], label='Training loss')
plt.plot(losses['validation'], label='Validation loss')
plt.legend()
_ = plt.ylim()

## model graph

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))

mean, std = scaled_features['cnt']
predictions = network.run(test_features).T*std + mean
ax.plot(predictions[0], label='Prediction')
ax.plot((test_targets['cnt']*std + mean).values, label='Data')
ax.set_xlim(right=len(predictions))
ax.legend()

dates = pd.to_datetime(rides.ix[test_data.index]['dteday'])
dates = dates.apply(lambda d: d.strftime('%b %d'))
ax.set_xticks(np.arange(len(dates))[12::24])
_ = ax.set_xticklabels(dates[12::24], rotation=45)